In [ ]:
import openai
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key"
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_prompt_type(section_name: str) -> int:
    """
    Returns prompt type based on section name:
    1: Beneficiary and Problem Statement
    2: Application Process and Benefits
    3: Outcome and Impact
    """
    if "Beneficiary and Problem Statement" in section_name:
        return 1
    elif "Application Process and Benefits" in section_name:
        return 2 
    elif "Outcome and Impact" in section_name:
        return 3
    return 1 

def convert_to_image_prompt(summary: str,context: str,character: str) -> str:
    """
    Converts a text summary into an image prompt using OpenAI's API.
    
    Args:
    - summary (str): The text summary of the welfare scheme section.
    
    Returns:
    - str: The generated image prompt.
    """
    base_prompts = {
        1: "Create a promotional poster highlighting the benefits and problems faced by the people in need. Show people in need of the welfare scheme actively receiving/using the benefits:",
        2: "Create a poster introducing the welfare scheme to its target beneficiaries. Show the problems being addressed:",
        3: "Create an impactful poster showing the positive outcomes and transformations. Show before/after scenarios:"
    }

    context = context.strip()
    selected_base = base_prompts.get(get_prompt_type(context), base_prompts[1])
    
    prompt = (
        f"{selected_base}\n\n"
        f"Here is a very good example for how to create a suitable prompt for such welfare schemes. Follow it's way of describing numerics and text: {context}\n"
        f"Character Profile: {character}\n"
        f"Summary: {summary}\n\n"
        "Generate a detailed visual description for the poster:"
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who has to convert a text summary into an image prompt (for a poster) for a text-to-image generative model. This generative model will be used to create a poster for the welfare scheme mentioned. Do not add text to the poster, this would make the poster more accessible to people who cannot read. ENSURE THAT THE PROMPT IS COMPLIANT WITH OPENAI'S POLICIES AND DOES NOT CONTAIN OFFENSIVE INFORMATION."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        n=1,
        stop=None,
        temperature=0.7
    )

    image_prompt = response.choices[0].message.content
    return image_prompt


In [2]:
import os

summaries_directory = "scheme_writeups_gpt"
image_prompts_directory = "image_prompts_prism"
context_dir = "image_prompt_context"

os.makedirs(image_prompts_directory, exist_ok=True)

def get_character_context(scheme, context_dir):
    """Get character context for a scheme."""
    context_file = os.path.join(context_dir, f"{scheme}.txt")
    if os.path.exists(context_file):
        with open(context_file, 'r') as f:
            return f.read().strip()
    return ""

def read_from_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
    return content

def generate_image_prompts_from_summaries(summaries_dir, prompts_dir):
    scheme_prompts = {}

    for filename in os.listdir(summaries_dir):
        if filename.endswith(".txt"):
            
            scheme_name, section_name = filename.split('_', 1)
            section_name = section_name.replace('.txt', '').replace('_', ' ')
            
            file_path = os.path.join(summaries_dir, filename)
            
            with open(file_path, "r") as summary_file:
                summary_content = summary_file.read()
                
            if scheme_name not in scheme_prompts:
                scheme_prompts[scheme_name] = []
            scheme_prompts[scheme_name].append((section_name, summary_content))
    
    for scheme_name, sections in scheme_prompts.items():
        for section, content in sections:
            character_context = get_character_context(scheme_name, context_dir)
            context = read_from_file("src/Best_Prompt_Complete")
            image_prompt = convert_to_image_prompt(content,section,character_context) 
            image_prompt = f"Create a poster with minimal text with the image from this image prompt : {image_prompt}"       
            prompt_filename = f"{scheme_name}_{section.replace(' ', '_')}.txt"
            prompt_file_path = os.path.join(prompts_dir, prompt_filename)
            
            with open(prompt_file_path, "w") as prompt_file:
                prompt_file.write(image_prompt)
                
            print(f"Image prompt saved for scheme: {scheme_name}, section: {section} as {prompt_filename}")

generate_image_prompts_from_summaries(summaries_directory, image_prompts_directory)

Image prompt saved for scheme: MGNREGA, section: Beneficiary and Problem Statement as MGNREGA_Beneficiary_and_Problem_Statement.txt
Image prompt saved for scheme: MGNREGA, section: Application Process and Benefits as MGNREGA_Application_Process_and_Benefits.txt
Image prompt saved for scheme: MGNREGA, section: Outcome and Impact as MGNREGA_Outcome_and_Impact.txt
Image prompt saved for scheme: ABPMJAY, section: Outcome and Impact as ABPMJAY_Outcome_and_Impact.txt
Image prompt saved for scheme: ABPMJAY, section: Application Process and Benefits as ABPMJAY_Application_Process_and_Benefits.txt
Image prompt saved for scheme: ABPMJAY, section: Beneficiary and Problem Statement as ABPMJAY_Beneficiary_and_Problem_Statement.txt
Image prompt saved for scheme: NRLM, section: Application Process and Benefits as NRLM_Application_Process_and_Benefits.txt
Image prompt saved for scheme: NRLM, section: Beneficiary and Problem Statement as NRLM_Beneficiary_and_Problem_Statement.txt
Image prompt saved for